<center><img src="MKn_Staffelter_Hof.jpeg" alt="Picture of old business"</center>
<!--Image Credit: Martin Kraft https://commons.wikimedia.org/wiki/File:MKn_Staffelter_Hof.jpg -->

Staffelter Hof Winery is Germany's oldest business, established in 862 under the Carolingian dynasty. It has continued to serve customers through dramatic changes in Europe, such as the Holy Roman Empire, the Ottoman Empire, and both world wars. What characteristics enable a business to stand the test of time?

To help answer this question, BusinessFinancing.co.uk researched the oldest company still in business in **almost** every country and compiled the results into several CSV files. This dataset has been cleaned.

Having useful information in different files is a common problem. While it's better to keep different types of data separate for data storage, you'll want all the data in one place for analysis. You'll use joining and data manipulation to work with this data and better understand the world's oldest businesses.

## The Data
`businesses` and `new_businesses`
|Column|Description|
|------|-----------|
|`business`|Name of the business (varchar)|
|`year_founded`|Year the business was founded (int)|
|`category_code`|Code for the business category (varchar)|
|`country_code`|ISO 3166-1 three-letter country code (char)|
---
`countries`
|Column|Description|
|------|-----------|
|`country_code`|ISO 3166-1 three-letter country code (varchar)|
|`country`|Name of the country (varchar)|
|`continent`|Name of the continent the country exists in (varchar)|
---
`categories`
|Column|Description|
|------|-----------|
|`category_code`|Code for the business category (varchar)|
|`category`|Description of the business category (varchar)|

In [62]:
-- What is the oldest business on each continent?
-- SELECT * FROM public.new_businesses
-- LIMIT 5;

-- SELECT c.continent, MIN(b.year_founded)
-- 		FROM public.countries as c INNER JOIN public.businesses as b
-- 		ON c.country_code = b.country_code
-- 		GROUP BY c.continent;

WITH CTE AS(
SELECT c.continent AS continent, c.country as country, b.business as business, b.year_founded as year_founded
FROM public.countries as c INNER JOIN public.businesses as b
ON c.country_code = b.country_code
	),
	cte2 as (
		SELECT c.continent as continent, MIN(b.year_founded) as min_year
		FROM public.countries as c INNER JOIN public.businesses as b
		ON c.country_code = b.country_code
		GROUP BY c.continent
	)
	
select CTE.continent, cte.country, cte.business, cte.year_founded 
FROM cte 
where cte.year_founded in (select cte2.min_year FROM cte2)
ORDER by cte.year_founded;

,continent,country,business,year_founded
0,Asia,Japan,Kongō Gumi,578
1,Europe,Austria,St. Peter Stifts Kulinarium,803
2,North America,Mexico,La Casa de Moneda de México,1534
3,South America,Peru,Casa Nacional de Moneda,1565
4,Africa,Mauritius,Mauritius Post,1772
5,Oceania,Australia,Australia Post,1809


In [2]:
-- How many countries per continent lack data on the oldest businesses
-- Does including the `new_businesses` data change this?

-- SELECT countries.continent, COUNT(countries.country) as count_nbc
-- FROM countries LEFT JOIN businesses ON
-- countries.country_code = businesses.country_code
-- WHERE businesses.business ISNULL
-- GROUP BY countries.continent;

-- SELECT COUNT(countries.country) as count_nbc
-- FROM countries LEFT JOIN businesses ON
-- countries.country_code = businesses.country_code
-- WHERE businesses.business ISNULL


-- WITH no_businesses_country AS (
-- SELECT countries.continent as continents, COUNT(countries.country) as count_nbc
-- FROM countries LEFT JOIN businesses ON
-- countries.country_code = businesses.country_code
-- WHERE businesses.business ISNULL
-- GROUP BY countries.continent
-- 	),
	
-- new_businesses_country AS (
-- 	SELECT countries.continent, COUNT(countries.country) as count_new_bc
-- 	FROM countries LEFT JOIN new_businesses ON
-- 	countries.country_code = new_businesses.country_code
-- 	WHERE new_businesses.business IS NOT NULL
-- 	GROUP BY countries.continent
-- ) 


-- SELECT (no_businesses_country.continents), (no_businesses_country.count_nbc - new_businesses_country.count_new_bc) as count_diff FROM no_businesses_country, new_businesses_country;

-- WITH no_businesses_country AS (
-- SELECT countries.continent AS nbc_continent, countries.country as nbc_country
-- FROM countries LEFT JOIN businesses ON
-- countries.country_code = businesses.country_code
-- WHERE businesses.business ISNULL
-- 	),
	
-- new_businesses_country AS (
-- 	SELECT countries.continent as new_bc_continent, countries.country as new_bc_country
-- 	FROM countries LEFT JOIN new_businesses ON
-- 	countries.country_code = new_businesses.country_code
-- 	WHERE new_businesses.business IS NOT NULL
-- ),

-- count_per_continent AS (
-- 	SELECT nbc_continent, nbc_country
-- 	FROM no_businesses_country
-- 	EXCEPT 
-- 	SELECT new_bc_continent, new_bc_country
-- 	FROM new_businesses_country 
-- 	)

-- SELECT count()


-- How many countries per continent lack data on the oldest businesses
-- Does including the `new_businesses` data change this?

-- SELECT countries.continent, COUNT(countries.country) as count_nbc
-- FROM countries LEFT JOIN businesses ON
-- countries.country_code = businesses.country_code
-- WHERE businesses.business ISNULL
-- GROUP BY countries.continent;

-- SELECT COUNT(countries.country) as count_nbc
-- FROM countries LEFT JOIN businesses ON
-- countries.country_code = businesses.country_code
-- WHERE businesses.business ISNULL




-- WITH cte AS (
-- SELECT countries.continent as continents, countries.country as country
-- FROM countries LEFT JOIN businesses ON
-- countries.country_code = businesses.country_code
-- WHERE businesses.business ISNULL
-- 	),
	
-- cte2 AS (
-- 	SELECT countries.country as new_bc_country
-- 	FROM countries LEFT JOIN new_businesses ON
-- 	countries.country_code = new_businesses.country_code
-- ) 


-- SELECT cte.continents, COUNT(cte.country)
-- FROM cte
-- WHERE cte.country IN (SELECT cte2.new_bc_country FROM cte2)
-- GROUP BY cte.continents;

WITH CTE AS (
	SELECT * FROM public.businesses
	UNION ALL
	SELECT * FROM public.new_businesses
)



SELECT countries.continent, COUNT(countries.country)
FROM countries LEFT JOIN cte 
ON countries.country_code = cte.country_code
WHERE cte.business ISNULL
GROUP BY countries.continent;





,continent,count
0,Africa,3
1,Asia,7
2,Europe,2
3,North America,5
4,Oceania,10
5,South America,3


In [3]:
-- Which business categories are best suited to last over the course of centuries?
WITH CTE AS (
	SELECT categories.category_code as cat_code, categories.category as all_categories
	FROM categories INNER JOIN businesses
	ON categories.category_code = businesses.category_code
	)

SELECT countries.continent, cte.all_categories, MIN(businesses.year_founded) as year_founded
FROM countries 
INNER JOIN businesses ON countries.country_code = businesses.country_code
INNER JOIN cte ON businesses.category_code = cat_code
GROUP BY countries.continent, cte.all_categories
ORDER BY year_founded, all_categories ASC;







,continent,all_categories,year_founded
0,Asia,Construction,578
1,Europe,"Cafés, Restaurants & Bars",803
2,Europe,"Distillers, Vintners, & Breweries",862
3,Europe,Manufacturing & Production,864
4,Asia,"Cafés, Restaurants & Bars",1153
5,Europe,Agriculture,1218
6,Europe,Tourism & Hotels,1230
7,Europe,Mining,1248
8,Europe,Medical,1422
9,Europe,Postal Service,1520


In [1]:
SELECT c.continent, COUNT(c.country_code) AS countries_without_businesses
FROM countries c
LEFT JOIN (
    SELECT * FROM businesses
    UNION ALL
    SELECT * FROM new_businesses
) b
ON c.country_code = b.country_code
WHERE b.business IS NULL
GROUP BY c.continent;

,continent,countries_without_businesses
0,Africa,3
1,Asia,7
2,Europe,2
3,North America,5
4,Oceania,10
5,South America,3


In [58]:
-- What is the oldest business on each continent?
SELECT bc.continent, bc.business, bc.year_founded, bc.country
FROM (
    SELECT business, year_founded, country, continent
    FROM businesses
    JOIN countries USING (country_code)
) bc
JOIN (SELECT continent, MIN(year_founded) AS year_founded
    FROM businesses
    JOIN countries 
    USING (country_code)
    GROUP BY continent
) c 
ON bc.continent = c.continent AND bc.year_founded = c.year_founded

,continent,business,year_founded,country
0,Oceania,Australia Post,1809,Australia
1,Europe,St. Peter Stifts Kulinarium,803,Austria
2,Asia,Kongō Gumi,578,Japan
3,North America,La Casa de Moneda de México,1534,Mexico
4,Africa,Mauritius Post,1772,Mauritius
5,South America,Casa Nacional de Moneda,1565,Peru


In [6]:
WITH no_businesses_country AS (
SELECT countries.continent as continents, COUNT(countries.country) as count_nbc
FROM countries LEFT JOIN businesses ON
countries.country_code = businesses.country_code
WHERE businesses.business ISNULL
GROUP BY countries.continent
	),
	
new_businesses_country AS (
	SELECT countries.continent, COUNT(countries.country) as count_new_bc
	FROM countries LEFT JOIN new_businesses ON
	countries.country_code = new_businesses.country_code
	GROUP BY countries.continent
) 


SELECT (no_businesses_country.continents), (no_businesses_country.count_nbc - new_businesses_country.count_new_bc) as count_diff FROM no_businesses_country, new_businesses_country;

,continents,count_diff
0,Africa,-50
1,Africa,-46
2,Africa,-9
3,Africa,-41
4,Africa,-20
5,Africa,-11
6,Asia,-46
7,Asia,-42
8,Asia,-5
9,Asia,-37
